In [1]:
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/CQR/

path = "./CANARD"


Mounted at /content/gdrive
/content/gdrive/MyDrive/CQR


In [2]:
def forget_history(l,k,max_length=480):
    title = l[0]
    section_title = l[1]
    k = int(min(k, (len(l)-2)/2))
    forget_index = int(len(l) - k*2)
    past_queries = [l[i] for i in range(2,forget_index,2)][::-1]
    near_queries = [l[i] for i in range(forget_index,len(l),2)][::-1]
    near_answers = [l[i] for i in range(forget_index+1,len(l),2)][::-1]
    context = []

    # Nombre tokens = nombre de mots
    length = len(title) + len(section_title)
    for i in range(len(near_queries)):
        length += len(near_queries[i]) + len(near_queries[i])
        if length >= max_length:
            break
        context.append(near_queries[i])
        context.append(near_answers[i])

    for i in range(len(past_queries)):
        length += len(past_queries[i])
        if length >= max_length:
            break
        context.append(past_queries[i])

    return [title,section_title] + context
    
def to_csv(df,dest):
    df = pd.DataFrame(df[["Question","History","Rewrite"]])
    df["Context"] = df["Question"] + " | " + df["History"]
    df.drop(columns=["Question","History"], inplace=True)
    df.to_csv(dest)
     

In [4]:
canard_train = pd.read_json(f'{path}/train.json').drop(columns=["QuAC_dialog_id","Question_no"])
canard_train["History"] = canard_train["History"].apply(lambda x: " | ".join(forget_history(x,k=np.inf)))
to_csv(canard_train,f'{path}/train_.csv')

canard_test = pd.read_json(f'{path}/test.json').drop(columns=["QuAC_dialog_id","Question_no"])
canard_test["History"] = canard_test["History"].apply(lambda x: " | ".join(forget_history(x,k=np.inf)))
to_csv(canard_test,f'{path}/test_.csv')

canard_dev = pd.read_json(f'{path}/dev.json').drop(columns=["QuAC_dialog_id","Question_no"])
canard_dev["History"] = canard_dev["History"].apply(lambda x: " | ".join(forget_history(x,k=np.inf)))
to_csv(canard_dev,f'{path}/dev_.csv')

raw = pd.read_json(f'{path}/test.json').drop(columns=["History","QuAC_dialog_id","Question_no"])
raw['Prediction'] = raw["Question"]
raw.drop(columns=["Question"])
raw = pd.DataFrame(raw[['Prediction','Rewrite']])
raw.to_csv(f'{path}/raw.csv', sep='\t')


In [ ]:
raw.head()

,Prediction,Rewrite
0,Did they have any clues?,Did investigators have any clues in the unreso...
1,How did they target her email?,How did FSB target the murdered journalist Ann...
2,Did they get into trouble for that?,Did FSB get into trouble for attacking the ema...
3,Did they have any murder suspects?,Did FSB have any Anna Politkovskaya's murder s...
4,Did they go to jail?,Did any of Anna Politkovskaya's murder suspect...
